# InSAR Processing Using ASF HYP3 on-demand products

## Getting Started

The Alaska Satellite Facility (ASF) archives SAR and InSAR data on behalf of NASA. 
By searching on their [Vertex website](https://search.asf.alaska.edu/#/?maxResults=250), users can access and download the original data, or in some cases can run basic processing. 
Vertex also allows users to do basic SBAS processing, which produces interferograms that can be downloaded and processed to time-series. 
This is what we will in this exercise. 


The ASF User guide can be found [here](https://docs.asf.alaska.edu/vertex/manual/)

See the HYP3 SDK [here](https://hyp3-docs.asf.alaska.edu/using/sdk/)

In [10]:
%pip install hyp3_sdk

  Using cached hyp3_sdk-7.7.1-py3-none-any.whl.metadata (5.5 kB)
Using cached hyp3_sdk-7.7.1-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from pathlib import Path

import numpy as np
import process_data
import rasterio as rio
from hyp3_sdk import HyP3


In [ ]:
hyp3 = HyP3(username=os.environ['HYP3_USERNAME'], password=os.environ['HYP3_PASSWORD'])
batch = hyp3.find_jobs()

Once you have complete jobs you can download the products to your machine

In [4]:
batch.complete()

True

### WARNING!!!
This step will download files from ASF. 
These files vary in size but are roughly ~0.5 GB each, so 200 will be ~100 GB. 
Make sure you have adequete space for downloading before starting! 
The good news is that file downloads don't repeat, so if the code stops for any reason you can restart from where it ended. 

In [2]:
# specify your download folder
dload_path_name = Path('asf_insar_data')

In [ ]:
batch.download_files(dload_path_name, create=True)

### Steps
The first steps are to download and unzip the products in our working directory. In this case, I have already done this step. 

In [3]:
%cd {dload_path_name}

[Errno 2] No such file or directory: 'asf_insar_data'
/mnt/g/Garlic/Documents/Code/GitHub/gemlab/software/insar


In [5]:
%ls

Cf_InSAR_and_GNSS_velocities.ipynb*  InSAR_with_ASF_ondemand_products.ipynb*
convert_h5_to_kml.py*                makeTS.py*
InSAR_start.ipynb*                   process_data.py*


In [ ]:
# Unzip your folders
# Depending on your platform this may not work
!unzip '*.zip'

UsageError: Line magic function `%unzip` not found.


In [ ]:
%ls

S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_48B4
S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_48B4.zip
S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_8B98
S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_8B98.zip
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_CDDA
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_CDDA.zip
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_E47F
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_E47F.zip
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E19F
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E19F.zip
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E943
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E943.zip
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_1989
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_1989.zip
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_2BF3
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_2

As you can see, we have a number of folders that each contain a set of files, including unwrapped phase, unwrapped displacements, DEM, look vectors, etc. 

Each folder is named by the two dates that were used to generate that interferogram, so for example `S1AA_20220420T235621_20220514T235622_VVP024_INT80_G_ueF_CF44` is an interferogram for dates 4/20/2022 and 5/14/2024. 

In [ ]:
# Count the number of interferograms we have downloaded
# If this line does not work on your machine feel free to skip
%ls -ld */ | wc -l  # this will count folders only

772


In [6]:
%mkdir zipped

In [7]:
%mv *.zip zipped/.

mv: cannot stat '*.zip': No such file or directory


In [ ]:
%ls

S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_48B4
S1AA_20200101T235556_20200113T235556_VVP012_INT80_G_ueF_8B98
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_CDDA
S1AA_20200101T235556_20200125T235555_VVP024_INT80_G_ueF_E47F
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E19F
S1AA_20200101T235556_20200206T235555_VVP036_INT80_G_ueF_E943
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_1989
S1AA_20200101T235621_20200113T235620_VVP012_INT80_G_ueF_2BF3
S1AA_20200101T235621_20200125T235620_VVP024_INT80_G_ueF_5CEF
S1AA_20200101T235621_20200125T235620_VVP024_INT80_G_ueF_F4E1
S1AA_20200101T235621_20200206T235620_VVP036_INT80_G_ueF_4219
S1AA_20200101T235621_20200206T235620_VVP036_INT80_G_ueF_8E59
S1AA_20200113T235556_20200125T235555_VVP012_INT80_G_ueF_5889
S1AA_20200113T235556_20200125T235555_VVP012_INT80_G_ueF_B72F
S1AA_20200113T235556_20200206T235555_VVP024_INT80_G_ueF_346D
S1AA_20200113T235556_20200206T235555_VVP024_INT80_G_ueF_C673
S1AA_20200113T235556_202

In [ ]:
# some QA/QC
%ls S1*/*_unw_phase.tif | wc -l

771


In [ ]:
# this should match the previous
%ls S1*/*_corr.tif | wc -l 

771


In [ ]:
# check file sizes
unw_paths = list(dload_path_name.glob('S1*/*_unw_phase.tif'))
corr_paths = list(dload_path_name.glob('S1*/*corr.tif'))
print(len(unw_paths))
print(len(corr_paths))

all_heights: list[float] = []
all_widths: list[float] = []
for path in unw_paths:
    with rio.open(path) as f:
        all_heights.append(f.height)
        all_widths.append(f.width)

print(np.mean(all_heights))
print(np.mean(all_widths))

771
771
2777.3813229571983
3538.284046692607


In [ ]:
assert len(set(unw_paths)) == len(unw_paths)

The next step is to make sure that all the interferograms have the same shape and geographic extent. 
We will ensure this using a simple code that uses rasterio to re-project all the interferograms to match a reference. 
For this code, you will need `rasterio` and `numpy` installed. 

In [ ]:
# for k, f in enumerate(files):
#     with rio.open(f) as F:
#         plt.plot([k,k], F.

In [ ]:
# The function to resize the images is called run_resampling
shp_file_path = Path('~/CEF/Projects/SLU/t92f120/asf_insar_data/stl_polygon/stl_polygon.shp')
process_data.run_resampling(shapefile_path=shp_file_path)  # runs on the current directory by default

ERROR 1: PROJ: proj_create_from_database: Open of /home/jlmd9g/.conda/envs/mintpy/share/proj failed
  0%|          | 1/771 [00:04<56:00,  4.36s/it]

Currently running file #0 of 771.


  1%|▏         | 11/771 [00:45<49:29,  3.91s/it]

Currently running file #10 of 771.


  3%|▎         | 21/771 [01:25<49:25,  3.95s/it]

...

Currently running file #750 of 771.


 99%|█████████▊| 761/771 [1:49:40<01:27,  8.74s/it]

Currently running file #760 of 771.


100%|██████████| 771/771 [1:51:18<00:00,  8.66s/it]

Currently running file #770 of 771.


Once the resampling has finished, we can now proceed with time-series analysis as normal!
You can refer to the [InSAR lab](https://github.com/jlmaurer/GE6146/blob/master/notebooks/Lab4_InSAR.ipynb) for more details and plotting examples. 

In [ ]:
# We'll use the same functions as for the lab
import makeTS

In [ ]:
dir = Path.cwd()
ifg_paths = list(dir.glob('**/*_unw_phase.tif'))
coh_paths = list(dir.glob('**/*_corr.tif'))

In [ ]:
print(f'I found {len(ifg_paths)} interferograms')

I found 771 interferograms


In [ ]:
# Pull all the data
date_pairs, dates = makeTS.get_dates(ifg_paths)
year_fracs = np.array([makeTS.get_year_frac(d) for d in dates])
g_matrix = makeTS.make_g_matrix(dates, date_pairs)
x_size, y_size, data_type, geo_proj, transform, nodata_val, num_bands = makeTS.read_raster(ifg_paths[0])
data = makeTS.get_data(ifg_paths, 1)

In [ ]:
print(f'Interferograms are {y_size}x{x_size} pixels in size')

Interferograms are 2841x3557 pixels in size


In [ ]:
# You'll need to make a plot of one of the dates or the average coherence so you can pick a reference point

In [ ]:
# De-referencing
data = makeTS.dereference(data, ref_center=[2500, 2000], ref_size=10)

In [ ]:
# Create the time-series data cube
ts_array = makeTS.make_ts(g_matrix, data, year_fracs)

In [ ]:
# Convert to displacement in meters
ts_array = makeTS.radians_to_meters(ts_array, lam=0.056)  # wavelength lam is 5.6 cm for Sentinel-1

In [ ]:
# Compute the mean velocity by fitting a line
vel = makeTS.find_mean_vel(ts_array, year_fracs)

In [11]:
vel.shape

(2841, 3557)

### Saving Data
Want to change this to use NETCDF and xarray, include time-series and coherence time-series in the files, maybe IFGs and coherence ifgs as well. 

In [ ]:
import h5py


with h5py.File('velocity.h5') as path:
    for key in path.keys():
        print(key)

intercept
interceptStd
residue
velocity
velocityStd


## Tropospheric correction using RAiDER

TBD